In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex)

# Other imports
import importlib

import scipy.interpolate

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [ ]:
## Pre-finalize steps
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = 0#np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles,
              stiffness=2.5e-2,
              damping=7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies())
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.get_body(BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    int(finger_instance),
    F_Nd=2,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

In [ ]:
vis.stop_recording()
vis.publish_recording()

In [ ]:
# Plot manipulator position vs. trajectory
# PROGRAMMING: Use body index for finger instead of model instance
if type(finger_ctrlr) is finger.PDFinger:
    plt.figure(figsize=(2*3,2*2))
    plt.plot(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1],
             logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2],
             label='Manipulator position')
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.legend()
    plt.show()

In [ ]:
# Generate plots for paper
nb = plant.num_bodies()
y_traces = []
z_traces = []
theta_traces = []
for b in paper.link_idxs:
    y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
    z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
    theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
y_traces = np.array(y_traces)
z_traces = np.array(z_traces)
theta_traces = np.array(theta_traces)

times = np.arange(0,effective_tspan, effective_tspan/10)
cmap = cm.get_cmap("viridis_r")
plt.figure(figsize=(2*3,2*2))
if type(finger_ctrlr) is finger.PDFinger:
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
for t in times:
    c = cmap(t/effective_tspan)
    idx = np.argmax(logger.sample_times() >= t)
    
    # Plot paper
    # PROGRAMMING: Account for paper thickness in plots
    for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
        y0 = y - np.cos(theta)*paper.link_width/2
        z0 = z - np.sin(theta)*paper.link_width/2
        y1 = y + np.cos(theta)*paper.link_width/2
        z1 = z + np.sin(theta)*paper.link_width/2
        plt.plot([y0, y1], [z0, z1], color=c)
        
    
    # Plot manipulator
    plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                color=c, s=300, zorder=1)

xlim = plt.xlim()
ylim = plt.ylim()
plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
plt.xlim(xlim)
plt.ylim(ylim)
cb = plt.colorbar()
cb.set_label("Time")
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.show()

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
# Plot drag forces
plt.figure(figsize=(16, 12))
nb = plant.num_bodies
vel_traces = []
for b in paper.link_idxs:
    rho = 1.225
    C_d = paper.link_width*paper.depth
    vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
    f = 0.5*rho*C_d*vel_sqred
    plt.plot(f)

plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
plt.show()

# Debug plots

In [ ]:
# Get debug dict
debug = finger_ctrlr.debug
times = np.arange(len(debug['d_Ns']))*effective_tspan/len(debug['d_Ns'])

## Unit vectors

In [ ]:
N_Hats = np.array(debug['N_hats'])
T_Hats = np.array(debug['T_hats'])
plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot(times, N_Hats[:,0,:], label='$\hat N_x$', color='red')
plt.plot(times, N_Hats[:,1,:], label='$\hat N_y$', color='green')
plt.plot(times, N_Hats[:,2,:], label='$\hat N_z$', color='blue')
plt.xlabel("Time (seconds)")
plt.legend()
plt.subplot(212)
plt.plot(times, T_Hats[:,0,:], label='$\hat T_x$', color='red')
plt.plot(times, T_Hats[:,1,:], label='$\hat T_y$', color='green')
plt.plot(times, T_Hats[:,2,:], label='$\hat T_z$', color='blue')
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

### Projection matrices
Gives a $k\times 3 \times 3$ array that can be used with `matmul` and vectors stacked in a $k\times 3 \times 1$ shape.

In [ ]:
# Transpose with [0, 2, 1] swaps only the last two rows
N_hat_proj_mats = np.matmul(N_Hats, np.transpose(N_Hats, [0, 2, 1]))
T_hat_proj_mats = np.matmul(T_Hats, np.transpose(N_Hats, [0, 2, 1]))

## Distance between link and manipulator

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['d_Ns'], label='$d_N$')
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.show()

## Link state

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['omega_xs'], label='$\omega_x$')
plt.plot(times, np.array(debug['theta_xs']), label='$\\theta_x$')
plt.legend()
plt.ylim(-2*np.pi, 2*np.pi)
plt.yticks(np.arange(-2, 2.5, 0.5)*np.pi, [
    '$-2\pi$',
    '$-3\pi/2$',
    '$-\pi$',
    '$-\pi/2$',
    '$0$',
    '$\pi/2$',
    '$\pi$',
    '$3\pi/2$',
    '$2\pi$',
])
plt.xlabel("Time (seconds)")
plt.axhline(np.pi, linestyle='--', color='k')
plt.axhline(-np.pi, linestyle='--', color='k')
plt.show()

## Forces

In [ ]:
# Friction force on manipulor, assuming static friction
total_m_FF = np.array(debug['F_GTs'])+np.array(debug['F_OTs'])-np.array(debug['F_CTs'])
# Total force on the link in the N hat direction 
total_l_FN = np.array(debug['F_Ns'])+np.array(debug['F_GNs'])+np.array(debug['F_ONs'])

### $\hat N$ acceleration
Take the acceleration in the $\hat N$ direction and then multiply by the mass. This is a check that the forces I'm summing in the $\hat N$ direction are actually all the forces on the object.

In [ ]:
ll_idx = paper.get_free_edge_idx()
a_vecs_no_interp = logger.data()[log_wrapper.entries_per_body*ll_idx+12:log_wrapper.entries_per_body*ll_idx+15].T
a_vecs_no_interp = np.expand_dims(a_vecs_no_interp, 2)

In [ ]:
a_vecs = scipy.interpolate.interp1d(logger.sample_times(), a_vecs_no_interp, axis=0)(times)
a_N = np.matmul(N_hat_proj_mats[:a_vecs.shape[0]], a_vecs)
F_from_acc = paper.link_mass*np.sum(a_N, axis=1)

### Tangential

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Tangential forces")
plt.plot(times, debug['F_GTs'], label='$F_{GT}$')
plt.plot(times, debug['F_OTs'], label='$F_{OT}$')
plt.plot(times, debug['F_CTs'], label='$F_{CT}$')
plt.plot(times, total_m_FF, label='Total $F_F$ (assuming static friction)')
plt.plot(times, -2*constants.FRICTION*np.array(debug['F_Ns']), 'k--', label="Bound on friction force")
plt.plot(times, 2*constants.FRICTION*np.array(debug['F_Ns']), 'k--')
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("$\hat T$ force (N)")
plt.show()

# PROGRAMMING: Let's get the actual contact forces, and possibly also compare the tangential acceleration, on this plot.

### Normal

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Normal forces")
plt.plot(times, debug['F_CNs'], label='$F_{CN}$')
plt.plot(times, debug['F_GNs'], label='$F_{GN}$')
plt.plot(times, debug['F_ONs'], label='$F_{ON}$')
plt.plot(times, total_l_FN, label='Total $\hat N$ force on link')
plt.plot(times, F_from_acc, label='Total force, according to acceleration')
plt.axhline(finger_ctrlr.F_Nd, color='k', linestyle='--', label='Target force')
plt.ylim(-finger_ctrlr.F_Nd*2, finger_ctrlr.F_Nd*2)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("$\hat N$ force (N)")
plt.show()

# PROGRAMMING: Why is the force not what my setpoint is?

## Control forces

### Link frame

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['F_CTs'], label='$F_{CT}$')
plt.plot(times, debug['F_CNs'], label='$F_{NT}$')
plt.legend()
plt.xlabel("Time (seconds)")
plt.show()

### Manipulator frame

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(times, debug['theta_xs'], label='$\\theta_x$')
plt.plot(times, debug['omega_xs'], label='$\omega_x$')
plt.legend()
plt.show()

In [ ]:
F_Ms = np.array(debug['F_Ms'])
plt.figure(figsize=(12, 12))
plt.plot(times, F_Ms[:,0,:], label='$F_M$ $x$ component', color='red')
plt.plot(times, F_Ms[:,1,:], label='$F_M$ $y$ component', color='green')
plt.plot(times, F_Ms[:,2,:], label='$F_M$ $z$ component', color='blue')
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()